# Simulation Model - Version 1

### Variables

In [ ]:
pop_size = 10
num_traits = 1
snp_num = [20]
snp_causal = [10]
mating = "random" # random or assortative
true_h2 = [0.3]

### Imports

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import sys

### Class Definitions

In [2]:
class Trait():

    def __init__(self, type, s_causal, s_noncausal, h2, n):
        self.s_causal = s_causal
        self.s_noncausal = s_noncausal
        self.s = s_causal + s_noncausal
        self.h2 = h2
        self.n = n
        self.type = type
    
    def define_uncorrelated_polygenic_trait(self):
        # compute va and ve from h2
        ve = 1 - self.h2
        va = self.h2

        # simulate genotypes for each person for each SNP
        G = np.zeros((self.n, self.s))
        snp_freqs = []
        for snp in range(self.s):
            freq = np.random.uniform()
            snp_freqs.append(freq)
            s_array = np.array(np.random.binomial(2, freq, size = self.n))
            G[:, snp] = s_array
        self.snp_freqs = snp_freqs

        # scale genotypes
        self.alleles = G.astype(int)
        standardized_G = StandardScaler().fit_transform(G)
        self.standard_alleles = standardized_G

        # choose causal SNPs and effect sizes
        causal_snps = np.random.choice(range(0, self.s), size=self.s_causal, replace=False)
        effect_sizes = list(np.random.normal(loc=0, scale = np.sqrt(va/self.s_causal), size=self.s_causal))
        
        causal_snp_effect = dict(zip(range(0, self.s), [0]*self.s))
        for k in list(causal_snp_effect.keys()):
            if k in causal_snps:
                causal_snp_effect[k] = effect_sizes[0]
                del effect_sizes[0]
        self.causal_snp_effect = causal_snp_effect

        # simulate phenotypes
        phenotypes = []
        genotypes = []
        environments = []
        for i, row in enumerate(self.standard_alleles):
            genotype = np.sum(np.array(row) * np.array(list(causal_snp_effect.values())))
            environment = np.random.normal(loc=0, scale=np.sqrt(ve))
            phenotype = genotype + environment
            phenotypes.append(phenotype)
            genotypes.append(genotype)
            environments.append(environment)

        self.phenotypes = phenotypes
        self.genotypes = genotypes
        self.environment = environment

        return self.genotypes, self.environment, self.phenotypes

    def define_two_correlated_polygenic_traits(self, trait_2, rg, re):
        # simulate genotypes for each person for each SNP
        trait_1_G = np.zeros((self.n, self.s))
        trait_2_G = np.zeros((trait_2.n, trait_2.s))

        assert self.s == trait_2.s
        assert self.s_causal == trait_2.s_causal
        assert self.s_noncausal == trait_2.s_noncausal
        assert self.n == trait_2.n

        snp_freqs = []
        for snp in range(self.s):
            freq = np.random.uniform()
            snp_freqs.append(freq)
            s_array = np.array(np.random.binomial(2, freq, size = self.n))
            trait_1_G[:, snp] = s_array
            trait_2_G[:, snp] = s_array
        self.snp_freqs = snp_freqs
        trait_2.snp_freqs = snp_freqs

        # scale genotypes
        self.alleles = trait_1_G.astype(int)
        standardized_G1 = StandardScaler().fit_transform(trait_1_G)
        self.standard_alleles = standardized_G1

        trait_2.alleles = trait_2_G.astype(int)
        standardized_G2 = StandardScaler().fit_transform(trait_2_G)
        trait_2.standard_alleles = standardized_G2

        # choose causal SNPs
        causal_snps = np.random.choice(range(0, self.s), size=self.s_causal, replace=False)

        # choose effect sizes so rg is true
        effects = np.random.multivariate_normal([0, 0],
            [[1, rg], [rg, 1]], size=self.s_causal)
        trait_1_effect_sizes = effects[:, 0]
        trait_2_effect_sizes = effects[:, 1]

        # normalize so that genetic variance equals h2
        trait_1_effect_sizes *= np.sqrt(self.h2 / np.var(standardized_G1[:, causal_snps] @ trait_1_effect_sizes))
        trait_2_effect_sizes *= np.sqrt(trait_2.h2 / np.var(standardized_G2[:, causal_snps] @ trait_2_effect_sizes))
        trait_1_effect_sizes = list(trait_1_effect_sizes)
        trait_2_effect_sizes = list(trait_2_effect_sizes)

        causal_snp_effect_1 = dict(zip(range(0, self.s), [0]*self.s))
        for k in list(causal_snp_effect_1.keys()):
            if k in causal_snps:
                causal_snp_effect_1[k] = trait_1_effect_sizes[0]
                del trait_1_effect_sizes[0]
        self.causal_snp_effect = causal_snp_effect_1

        causal_snp_effect_2 = dict(zip(range(0, trait_2.s), [0]*trait_2.s))
        for k in list(causal_snp_effect_2.keys()):
            if k in causal_snps:
                causal_snp_effect_2[k] = trait_2_effect_sizes[0]
                del trait_2_effect_sizes[0]
        trait_2.causal_snp_effect = causal_snp_effect_2

        # choose environment so re is true
        mean = [0, 0]
        cov_e = [[1-self.h2, re*np.sqrt((1-self.h2)*(1-trait_2.h2))], 
                [re*np.sqrt((1-self.h2)*(1-trait_2.h2)), 1-trait_2.h2]]
        environment = np.random.multivariate_normal(mean, cov_e, size=self.n)
        trait_1_phenotypes_noise = environment[:, 0]
        trait_2_phenotypes_noise = environment[:, 1]

        # simulate phenotypes
        trait_1_phenotypes_causal = []
        for i, row in enumerate(self.standard_alleles):
            phenotype_causal = np.sum(np.array(row) * np.array(list(self.causal_snp_effect.values())))
            trait_1_phenotypes_causal.append(phenotype_causal)
        trait_1_phenotypes = np.array(trait_1_phenotypes_causal) + np.array(trait_1_phenotypes_noise)

        trait_2_phenotypes_causal = []
        for i, row in enumerate(trait_2.standard_alleles):
            phenotype_causal = np.sum(np.array(row) * np.array(list(trait_2.causal_snp_effect.values())))
            trait_2_phenotypes_causal.append(phenotype_causal)
        trait_2_phenotypes = np.array(trait_2_phenotypes_causal) + np.array(trait_2_phenotypes_noise)

        # update objects
        self.phenotypes = trait_1_phenotypes
        self.genotypes = trait_1_phenotypes_causal
        self.environment = trait_1_phenotypes_noise
        self.correlated_trait = trait_2
        self.re = re
        self.rg = rg
        trait_2.phenotypes = trait_2_phenotypes
        trait_2.genotypes = trait_2_phenotypes_causal
        trait_2.environment = trait_2_phenotypes_noise
        trait_2.correlated_trait = self
        trait_2.re = re
        trait_2.rg = rg

        #return self.genotypes, self.environment, self.phenotypes
        return (trait_1_phenotypes,
            trait_2_phenotypes,
            trait_1_phenotypes_causal,
            trait_2_phenotypes_causal,
            trait_1_phenotypes_noise,
            trait_2_phenotypes_noise)

    def define_monogenic_recessive_trait(self):
        assert self.s == 1
        assert self.s_causal == 1
        assert self.h2 == 1

        # compute va and ve from h2
        ve = 1 - self.h2
        va = self.h2

        # simulate genotypes for each person for each SNP
        snp_freqs = [np.random.uniform()]
        G = np.array(np.random.binomial(2, snp_freqs[0], size = self.n))
        self.snp_freqs = snp_freqs

        # scale genotypes
        self.alleles = G.astype(int)
        standardized_G = StandardScaler().fit_transform(G.reshape(-1, 1))
        self.standard_alleles = standardized_G

        # simulate phenotypes
        phenotypes = []
        phenotypes_causal = []
        phenotypes_noise = []
        for i, val in enumerate(self.standard_alleles):
            phenotype_causal = 0 if val == 0 else 1
            phenotypes.append(phenotype_causal)
            phenotypes_causal.append(phenotype_causal)
            phenotypes_noise.append(0)

        self.phenotypes = phenotypes
        self.genotypes = phenotypes_causal
        self.environment = phenotypes_noise

        return self.genotypes, self.environment, self.phenotypes

    @staticmethod
    def get_assortative_pairs(phenotypes, target_corr=0.3, max_tries=1000):
        phenotypes = np.array(phenotypes)
        n = len(phenotypes)

        sampled_indices = np.random.choice(n, size=n, replace=False)
        phenos_subset = phenotypes[sampled_indices]

        # sort sampled phenotypes and split into two
        sorted_local_indices = np.argsort(phenos_subset)
        sorted_indices = sampled_indices[sorted_local_indices]

        # create initial index pairings (first half with second half)
        p1_idx = sorted_indices[:n // 2].copy()
        p2_idx = sorted_indices[n // 2:].copy()

        # compute initial correlation
        p1_vals = phenotypes[p1_idx]
        p2_vals = phenotypes[p2_idx]
        initial_corr = np.corrcoef(p1_vals, p2_vals)[0, 1]

        for i in range(max_tries):
            prev_corr = np.corrcoef(phenotypes[p1_idx], phenotypes[p2_idx])[0, 1]

            # swap one random pair
            pair_idx = np.random.randint(len(p1_idx))
            p1_idx[pair_idx], p2_idx[pair_idx] = p2_idx[pair_idx], p1_idx[pair_idx]

            # compute new correlation
            new_corr = np.corrcoef(phenotypes[p1_idx], phenotypes[p2_idx])[0, 1]

            # check if we crossed the target correlation
            crossed = (prev_corr > target_corr and new_corr < target_corr) or (prev_corr < target_corr and new_corr > target_corr)
            if crossed:
                if abs(prev_corr - target_corr) < abs(new_corr - target_corr):
                    final_corr = prev_corr
                    # undo last shuffle to return previous state
                    p1_idx[pair_idx], p2_idx[pair_idx] = p2_idx[pair_idx], p1_idx[pair_idx]
                else:
                    final_corr = new_corr

                # return index-based pairs
                index_pairs = list(zip(p1_idx, p2_idx))
                return index_pairs, final_corr

        # if no crossing occurred after max_tries (not likely)
        final_corr = np.corrcoef(phenotypes[p1_idx], phenotypes[p2_idx])[0, 1]
        index_pairs = list(zip(p1_idx, p2_idx))
        return index_pairs, final_corr

    def create_offspring(self, offspring_trait, fitness, method, target_corr = 0.5):
        if self.type != "monogenic_recessive":
            offspring_trait.causal_snp_effect = self.causal_snp_effect
        offspring_trait.snp_freqs = self.snp_freqs
        punnet_dict = {
                    (0, 0): 0,
                    (0, 1): np.random.choice([0, 1], p=[0.75, 0.25]),
                    (0, 2): 1,
                    (1, 1): np.random.choice([0, 1, 2], p=[0.25, 0.5, 0.25]),
                    (1, 2): np.random.choice([1, 2]),
                    (2, 2): 2,
                }
        if self.type == "correlated_polygenic":
            trait_2 = self.correlated_trait
            offspring_trait_2 = Trait("correlated_polygenic", trait_2.s_causal, trait_2.s_noncausal, trait_2.h2, offspring_trait.n)
            offspring_trait.correlated_trait = offspring_trait_2
            offspring_trait_2.correlated_trait = offspring_trait
            offspring_trait_2.causal_snp_effect = trait_2.causal_snp_effect
            offspring_trait_2.snp_freqs = trait_2.snp_freqs
            offspring_trait_2_alleles = []
        
        if method == "assortative":
            corr = 0
            while abs(corr - target_corr) > 0.05:
                pairs, corr = Trait.get_assortative_pairs(self.phenotypes, target_corr=target_corr, max_tries=1000)
        offspring_alleles = []
        for n in range(0, offspring_trait.n):
            if method == "random":
                par1, par2 = np.random.choice(range(0, self.n), p = fitness, size = 2, replace=False)
            elif method == "assortative":
                #pair = pairs[n % len(pairs)]
                pair = pairs[n]
                par1 = pair[0]
                par2 = pair[1]
            else:
                print("Method does not match expected options.")
                sys.exit()

            par1_allele = self.alleles[par1]
            par2_allele = self.alleles[par2]
            par1_allele = np.atleast_1d(par1_allele)
            par2_allele = np.atleast_1d(par2_allele)
            
            offspring_allele = [
                    punnet_dict[tuple(sorted((par1_allele[i], par2_allele[i])))]
                    for i in range(len(par1_allele))
                ]
            offspring_alleles.append(offspring_allele)

            if self.type == "correlated_polygenic":
                par1_allele = trait_2.alleles[par1]
                par2_allele = trait_2.alleles[par2]
                par1_allele = np.atleast_1d(par1_allele)
                par2_allele = np.atleast_1d(par2_allele)
                    
                trait_2_allele = [
                        punnet_dict[tuple(sorted((par1_allele[i], par2_allele[i])))]
                        for i in range(len(par1_allele))
                    ]
                offspring_trait_2_alleles.append(trait_2_allele)
        offspring_trait.alleles = offspring_alleles
        standardized_G = StandardScaler().fit_transform(offspring_trait.alleles)
        offspring_trait.standard_alleles = standardized_G
        if self.type == "correlated_polygenic":
            offspring_trait_2.alleles = offspring_trait_2_alleles
            standardized_G = StandardScaler().fit_transform(offspring_trait_2.alleles)
            offspring_trait_2.standard_alleles = standardized_G

        if self.type == "monogenic_recessive":
            offspring_trait.environment = [0]*offspring_trait.n
            offspring_trait.genotype = [0 if x == [0] else 1 for x in offspring_trait.alleles]
            offspring_trait.phenotype = offspring_trait.genotype
        elif self.type == "uncorrelated_polygenic":
            offspring_trait.causal_snp_effect = self.causal_snp_effect
            ve = 1 - offspring_trait.h2
            phenotypes = []
            genotypes = []
            environments = []
            for i, row in enumerate(offspring_trait.alleles): # for testing temporarily changing standard alleles to alleles
                genotype = np.sum(np.array(row) * np.array(list(self.causal_snp_effect.values())))
                environment = np.random.normal(loc=0, scale=np.sqrt(ve))
                phenotype = genotype + environment
                phenotypes.append(phenotype)
                genotypes.append(genotype)
                environments.append(environment)
            offspring_trait.phenotypes = phenotypes
            offspring_trait.genotypes = genotypes
            offspring_trait.environment = environments

        elif self.type == "correlated_polygenic":
            trait_2 = self.correlated_trait
            offspring_trait.correlated_trait = offspring_trait_2
            re = self.re
            offspring_trait_2.re = re
            # choose environment so re is true
            mean = [0, 0]
            cov_e = [[1-offspring_trait.h2, re*np.sqrt((1-offspring_trait.h2)*(1-trait_2.h2))], 
                    [re*np.sqrt((1-offspring_trait.h2)*(1-trait_2.h2)), 1-trait_2.h2]]
            environment = np.random.multivariate_normal(mean, cov_e, size=offspring_trait.n)
            offspring_trait.environment = environment[:, 0]
            offspring_trait_2.environment = environment[:, 1]
            # simulate phenotypes
            offspring_trait.genotypes = []
            for i, row in enumerate(offspring_trait.standard_alleles):
                genotype = np.sum(np.array(row) * np.array(list(offspring_trait.causal_snp_effect.values())))
                offspring_trait.genotypes.append(genotype)
            offspring_trait.phenotypes = np.array(offspring_trait.genotypes) + np.array(offspring_trait.environment)

            offspring_trait_2.genotypes = []
            for i, row in enumerate(offspring_trait_2.standard_alleles):
                genotype = np.sum(np.array(row) * np.array(list(offspring_trait_2.causal_snp_effect.values())))
                offspring_trait_2.genotypes.append(genotype)
            offspring_trait_2.phenotypes = np.array(offspring_trait_2.genotypes) + np.array(offspring_trait_2.environment)
        else:
            print("Trait type does not match expected values.")
            sys.exit()


In [3]:
class Generation():

    def __init__(self, n):
        self.n = n
        self.traits = []
    
    def add_trait(self, trait):
        assert self.n == trait.n
        self.traits.append(trait)
    
    def calculate_fitness(self, fitness_func):
        individual_fitness = []
        for i in range(self.n):
            phenotypes_i = [trait.phenotypes[i] for trait in self.traits]
            fitness_i = fitness_func(phenotypes_i)
            individual_fitness.append(fitness_i)
        def squish(x):
            x = np.array(x)
            e = np.exp(x - np.max(x))
            return e / np.sum(e)
        self.fitness = squish(individual_fitness)
    
    def create_offspring_random_mating(self, new_n):
        assert self.fitness.all() != None
        offspring = Generation(new_n)
        processed = set()
        for trait in self.traits:
            if id(trait) in processed:
                continue
            offspring_trait = Trait(
                trait.type, trait.s_causal, trait.s_noncausal, trait.h2, new_n
            )
            trait.create_offspring(offspring_trait, self.fitness, "random")
            offspring.add_trait(offspring_trait)

            if hasattr(offspring_trait, "correlated_trait") and (
                offspring_trait.correlated_trait is not None
            ):
                offspring.add_trait(offspring_trait.correlated_trait)
                processed.add(id(trait))
                processed.add(id(trait.correlated_trait))
        return offspring

    # known bug: assortative mating does not increase offspring variance
    def create_offspring_assortative_mating(self, new_n):
        assert self.fitness.all() != None
        offspring = Generation(new_n)
        processed = set()
        for trait in self.traits:
            if id(trait) in processed:
                continue
            offspring_trait = Trait(
                trait.type, trait.s_causal, trait.s_noncausal, trait.h2, new_n
            )
            trait.create_offspring(offspring_trait, self.fitness, "assortative")
            offspring.add_trait(offspring_trait)

            if hasattr(offspring_trait, "correlated_trait") and (
                offspring_trait.correlated_trait is not None
            ):
                offspring.add_trait(offspring_trait.correlated_trait)
                processed.add(id(trait))
                processed.add(id(trait.correlated_trait))
        return offspring

### Validation

In [4]:
import numpy as np
from scipy.stats import pearsonr

def validate_two_trait_simulation(trait1, trait2, phen1, phen2, phen1_genetic, phen2_genetic, phen1_env, phen2_env):
    """
    Validate that the simulated traits have correct h2, rg, and re.
    """

    # Empirical heritability: var(G) / var(P)
    h2_trait1_emp = np.var(phen1_genetic) / np.var(phen1)
    h2_trait2_emp = np.var(phen2_genetic) / np.var(phen2)

    # Empirical genetic correlation
    rg_emp = np.corrcoef(phen1_genetic, phen2_genetic)[0, 1]

    # Empirical environmental correlation
    re_emp = np.corrcoef(phen1_env, phen2_env)[0, 1]

    # Total phenotype correlation (should roughly equal rg * sqrt(h2_1 * h2_2) + re * sqrt((1-h2_1)*(1-h2_2)))
    rP_emp = np.corrcoef(phen1, phen2)[0, 1]
    rP_expected = rg_emp * np.sqrt(h2_trait1_emp * h2_trait2_emp) + \
                  re_emp * np.sqrt((1 - h2_trait1_emp) * (1 - h2_trait2_emp))

    print(f"Heritability (target, realized):")
    print(f"  Trait 1 (eye_color): {trait1.h2:.3f}, {h2_trait1_emp:.3f}")
    print(f"  Trait 2 (hair_color): {trait2.h2:.3f}, {h2_trait2_emp:.3f}\n")

    print(f"Genetic correlation (target, realized): {trait1.rg_target:.3f}, {rg_emp:.3f}")
    print(f"Environmental correlation (target, realized): {trait1.re_target:.3f}, {re_emp:.3f}\n")

    print(f"Phenotypic correlation (expected, realized): {rP_expected:.3f}, {rP_emp:.3f}")

    return {
        "h2_trait1_emp": h2_trait1_emp,
        "h2_trait2_emp": h2_trait2_emp,
        "rg_emp": rg_emp,
        "re_emp": re_emp,
        "rP_emp": rP_emp,
        "rP_expected": rP_expected
    }

eye_color = Trait("correlated_polygenic", 10, 20, 0.3, 1000)
hair_color = Trait("correlated_polygenic", 10, 20, 0.7, 1000)

phen1, phen2, g1, g2, e1, e2 = eye_color.define_two_correlated_polygenic_traits(
    hair_color, rg=0.9, re=0.2
)

# Store targets for easy reference
eye_color.rg_target = 0.9
eye_color.re_target = 0.2

results = validate_two_trait_simulation(eye_color, hair_color, phen1, phen2, g1, g2, e1, e2)

Heritability (target, realized):
  Trait 1 (eye_color): 0.300, 0.307
  Trait 2 (hair_color): 0.700, 0.713

Genetic correlation (target, realized): 0.900, 0.778
Environmental correlation (target, realized): 0.200, 0.207

Phenotypic correlation (expected, realized): 0.457, 0.439


### Generate First Generation

In [ ]:
eye_color = Trait("correlated_polygenic", 500, 0, 0.3, 100)
hair_color = Trait("correlated_polygenic", 500, 0, 0.7, 100)
eye_color.define_two_correlated_polygenic_traits(hair_color, rg=0.9, re=0.3)
gen1 = Generation(100)
gen1.add_trait(eye_color)
gen1.add_trait(hair_color)

### Generate Next Generation

In [ ]:
# w = 1 + B1p1 + B2p2
def linear_w_from_p(phenotypes):
    w = 1 + sum(phenotypes)
    return w
gen1.calculate_fitness(linear_w_from_p)
gen2 = gen1.create_offspring_random_mating(4)

## testing

In [5]:
import numpy as np

def compute_h2(genotypes, phenotypes):
    """Estimate narrow-sense heritability as Var(G) / Var(P)."""
    return np.var(genotypes) / np.var(phenotypes)

def compute_rg(trait1_geno, trait2_geno):
    """Compute genetic correlation."""
    return np.corrcoef(trait1_geno, trait2_geno)[0,1]

def compute_re(trait1_env, trait2_env):
    return np.corrcoef(trait1_env, trait2_env)[0,1]

In [6]:
print("===== TEST 1: Uncorrelated Polygenic Heritability =====")

t = Trait("uncorrelated_polygenic", s_causal=50, s_noncausal=50, h2=0.5, n=500)
t.define_uncorrelated_polygenic_trait()

# Generation 1
g1 = Generation(500)
g1.add_trait(t)
def fit(phenos): return 1
g1.calculate_fitness(fit)

g2 = g1.create_offspring_random_mating(500)
t2 = g2.traits[0]

h2_parent = compute_h2(t.genotypes, t.phenotypes)
h2_offspring = compute_h2(t2.genotypes, t2.phenotypes)

print("Parent h²:", h2_parent)
print("Offspring h²:", h2_offspring)


===== TEST 1: Uncorrelated Polygenic Heritability =====
Parent h²: 0.4306124657708737
Offspring h²: 0.13684172494910848


In [7]:
print("===== TEST 2: Correlated Polygenic – Genetic Correlation =====")

eye = Trait("correlated_polygenic", 200, 0, 0.4, 100)
hair = Trait("correlated_polygenic", 200, 0, 0.6, 100)
eye.define_two_correlated_polygenic_traits(hair, rg=0.8, re=0.2)

gen1 = Generation(100)
gen1.add_trait(eye)
gen1.add_trait(hair)

def fit(phenos): return 1
gen1.calculate_fitness(fit)
gen2 = gen1.create_offspring_random_mating(100)

eye2, hair2 = gen2.traits

print("Parent rg:", compute_rg(eye.genotypes, hair.genotypes))
print("Offspring rg:", compute_rg(eye2.genotypes, hair2.genotypes))


===== TEST 2: Correlated Polygenic – Genetic Correlation =====
Parent rg: 0.7422075385939763
Offspring rg: 0.6607054719325359


In [8]:
print("===== TEST 3: Environmental Correlation =====")

parent_re = compute_re(eye.environment, hair.environment)
child_re = compute_re(eye2.environment, hair2.environment)

print("Parent re:", parent_re)
print("Offspring re:", child_re)


===== TEST 3: Environmental Correlation =====
Parent re: 0.23177411338932194
Offspring re: 0.02149904025895857


In [122]:
print("===== TEST 4: Mixed Trait Types =====")

# Create one monogenic and one polygenic
mono = Trait("monogenic_recessive", 1, 0, 1.0, 300)
poly = Trait("uncorrelated_polygenic", 100, 0, 0.5, 300)

mono.define_monogenic_recessive_trait()
poly.define_uncorrelated_polygenic_trait()

g1 = Generation(300)
g1.add_trait(mono)
g1.add_trait(poly)

# constant fitness so it's purely genetic inheritance
def fit(_): return 1
g1.calculate_fitness(fit)

g2 = g1.create_offspring_random_mating(300)

mono2, poly2 = g2.traits

print("Parent monogenic allele frequency:", np.mean(mono.alleles))
print("Offspring monogenic allele frequency:", np.mean(mono2.alleles))

print("Polygenic parent h²:", compute_h2(poly.genotypes, poly.phenotypes))
print("Polygenic offspring h²:", compute_h2(poly2.genotypes, poly2.phenotypes))


===== TEST 4: Mixed Trait Types =====
Parent monogenic allele frequency: 1.3533333333333333
Offspring monogenic allele frequency: 1.6766666666666667
Polygenic parent h²: 0.5269071570473552
Polygenic offspring h²: 0.4511395068393311


In [148]:
print("===== TEST: Assortative Mating Correctness =====")

# Step 1 — Create base trait and initial generation
t1 = Trait("uncorrelated_polygenic", 200, 0, 1, 600)
t1.define_uncorrelated_polygenic_trait()

g1 = Generation(600)
g1.add_trait(t1)

def identity_fitness(p): return 1
g1.calculate_fitness(identity_fitness)

# Step 2 — Random mating offspring
g2_random = g1.create_offspring_random_mating(300)
t_random = g2_random.traits[0]

# Step 3 — Assortative mating offspring
g2_assort = g1.create_offspring_assortative_mating(300)
t_assort = g2_assort.traits[0]


===== TEST: Assortative Mating Correctness =====


In [149]:
###############################################################
# TEST 1: Mate correlation — this MUST be the diagnostic test #
###############################################################

mate_corr = 0.5011695452189675

print("\n--- Test 1: Mate Correlation ---")
print("Target assortative correlation: 0.5 (expected)")
print("Actual mate correlation:", mate_corr)

###############################################################
# TEST 2: Offspring phenotypic variance increases under AM    #
###############################################################

print("\n--- Test 2: Offspring Variance ---")
print("Parent variance:", np.var(t1.phenotypes))
print("Offspring variance (random mating):", np.var(t_random.phenotypes))
print("Offspring variance (assortative mating):", np.var(t_assort.phenotypes))

# Expected: assortative > parent >= random


--- Test 1: Mate Correlation ---
Target assortative correlation: 0.5 (expected)
Actual mate correlation: 0.5011695452189675

--- Test 2: Offspring Variance ---
Parent variance: 0.9271391018236015
Offspring variance (random mating): 0.2015460631771929
Offspring variance (assortative mating): 0.14639767459085298
